In [24]:
from collections import Counter
from os import cpu_count

import joblib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from PIL import Image
from pytorch_lightning import seed_everything
from sklearn.cluster import DBSCAN
import cv2
from torchvision import transforms, utils
from torch.utils.data import DataLoader

from utils import pca_explained_variance_ratio
from metric import getMetric
from bert_embedding.bert_utility import tokenize_sentence, get_bert_embedding
from PostDataset import PostDataset

In [2]:
seed = 1
seed_everything(seed)

Global seed set to 1


1

In [20]:
train_df = pd.read_csv('dataset/train.csv')

In [21]:
tmp = train_df.groupby('label_group').posting_id.agg('unique').to_dict()
train_df['target'] = train_df.label_group.map(tmp)

### Data for BERT embedding

In [3]:
transforms = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

In [4]:
post_dataset = PostDataset(csv_file='dataset/train.csv', root_dir='./dataset/train_images/', transform=transforms)

In [5]:
post_dataloader = DataLoader(post_dataset, batch_size=1024, shuffle=False, num_workers=0)

### Get BERT emb of title

In [6]:
bert_embs = []

for i_batch, data in enumerate(post_dataloader):
#     print(i_batch, data['image'].shape, data['token_tensor'].shape, data['attn_mask'].shape)
    token_tensors = data['token_tensor'].cuda()
    attn_masks = data['attn_mask'].cuda()
    batch_embs = get_bert_embedding(token_tensors, attn_masks)
    bert_embs.append(batch_embs.cpu())

/home/chien/hdd/miniconda3/envs/shopee/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2068: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [17]:
embs = bert_embs[0]

for emb in bert_embs[1:]:
    embs = torch.cat((embs, emb), dim=0)
joblib.dump(embs, 'tmp/title_bert_embs.joblib')

In [26]:
title_embs = joblib.load('tmp/title_bert_embs.joblib')
title_embs.shape

torch.Size([34250, 768])

### Clustering

In [ ]:
bert_reduced_2 = UMAP(n_components=2, random_state=seed, transform_seed=seed).fit_transform(title_embs)
joblib.dump(bert_reduced_2, 'tmp/title_bert_embs_reduced_2.joblib')

# reduced_2 = joblib.load('tmp/tf_efficientnet_b8_reduced_2.joblib')
# plt.scatter(reduced_2[:, 0], reduced_2[:, 1])
# plt.tight_layout()
# plt.show()

In [27]:
labels = DBSCAN(eps=0.05, min_samples=2, n_jobs=-1).fit_predict(title_embs)
uniq_labels = list(sorted(set(labels)))

# print(Counter(labels))

# for label in uniq_labels:
#     mask = (labels == label)
#     plt.scatter(reduced_2[mask, 0], reduced_2[mask, 1])
# plt.legend(uniq_labels, bbox_to_anchor=(1.05, 1))
# plt.tight_layout()
# plt.show()

NameError: name 'reduced_2' is not defined

In [28]:
len(uniq_labels)

1369

In [29]:
train_df['labels'] = labels

In [31]:
tmp = train_df.groupby('labels').posting_id.agg('unique').to_dict()
train_df['oof'] = train_df.labels.map(tmp)

In [32]:
def outlier_group(row):
    if row.labels == -1:
        return [row.posting_id]
    else:
        return row.oof
train_df['oof'] = train_df.apply(outlier_group, axis=1)

### Calculate metric

In [34]:
train_df['f1'] = train_df.apply(getMetric('oof'), axis=1)
print('CV score =', train_df.f1.mean())

CV score = 0.4832022085558331
